In [1]:
!pip install llama-index==0.11.11 -q


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install docx2txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## 2.2 데이터 로딩

### 2.2.1 데이터 리더

In [20]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("sample_docs").load_data()

Ignoring wrong pointing object 6 0 (offset 0)


In [21]:
# Document 리스트의 각 요소에 접근하여 내용 출력
for i, document in enumerate(documents):
    print(f"Document {i+1}:")
    print(document.text)
    print("\n")

Document 1:
This is the content of file 1.


Document 2:
This is the content of file 2.


Document 3:
This is the content of ﬁle 3. 




In [22]:
documents = SimpleDirectoryReader(
    "sample_docs", recursive=True).load_data()

Ignoring wrong pointing object 6 0 (offset 0)


In [23]:
# Document 리스트의 각 요소에 접근하여 내용 출력
for i, document in enumerate(documents):
    print(f"Document {i+1}:")
    print(document.text)
    print("\n")


Document 1:
This is the content of file 1.


Document 2:
This is the content of file 2.


Document 3:
This is the content of ﬁle 3. 


Document 4:
This is the content of file 4.


Document 5:
This is the content of file 5.




In [24]:
documents = SimpleDirectoryReader(
    "sample_docs",
    required_exts=[".txt", ".pdf"],
    recursive=True).load_data()

Ignoring wrong pointing object 6 0 (offset 0)


In [25]:
# Document 리스트의 각 요소에 접근하여 내용 출력
for i, document in enumerate(documents):
    print(f"Document {i+1}:")
    print(document.text)
    print("\n")


Document 1:
This is the content of file 1.


Document 2:
This is the content of ﬁle 3. 


Document 3:
This is the content of file 4.




In [26]:
# document 리스트의 각 요소에 접근하여 메타데이터를 출력
for i, document in enumerate(documents):
    print(f"Document {i+1} metadata:")
    print(document.metadata)
    print("\n")

Document 1 metadata:
{'file_path': '/external/llama-index-tutorial/ch02/sample_docs/file1.txt', 'file_name': 'file1.txt', 'file_type': 'text/plain', 'file_size': 30, 'creation_date': '2025-07-04', 'last_modified_date': '2025-07-04'}


Document 2 metadata:
{'page_label': '1', 'file_name': 'file3.pdf', 'file_path': '/external/llama-index-tutorial/ch02/sample_docs/file3.pdf', 'file_type': 'application/pdf', 'file_size': 9283, 'creation_date': '2025-07-04', 'last_modified_date': '2025-07-04'}


Document 3 metadata:
{'file_path': '/external/llama-index-tutorial/ch02/sample_docs/sub_sample_docs/file4.txt', 'file_name': 'file4.txt', 'file_type': 'text/plain', 'file_size': 30, 'creation_date': '2025-07-04', 'last_modified_date': '2025-07-04'}




### 2.2.2 데이터 커넥터

In [27]:
!pip install llama-index-readers-database==0.3.0 -q


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [28]:
!pip install pymysql


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [64]:
from sqlalchemy import create_engine
from llama_index.readers.database import DatabaseReader

# MySQL 연결 정보 직접 입력
scheme = "mysql+pymysql"
host = "localhost"
password = "!23"
port = "3306"
user = "root"
dbname = "test_db"

connection_string = f"{scheme}://{user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(connection_string)
reader = DatabaseReader(sql_database=engine)

# 데이터 로드
query = "SELECT * FROM users"
documents = reader.load_data(query=query)


In [65]:
# documents 리스트의 각 요소 출력
for idx, doc in enumerate(documents):
    print(f"Document {idx + 1}:")
    print(f"ID {doc.id_}")
    print(f"Row {doc.text}")
    print("-" * 50)


Document 1:
ID 86b1b2ca-60d7-48d5-af26-b90eb7ea9893
Row id: 1, name: Alice, email: alice@example.com, age: 30
--------------------------------------------------
Document 2:
ID d1734329-0aa1-4694-b892-d644590c0e26
Row id: 2, name: Bob, email: bob@example.com, age: 25
--------------------------------------------------
Document 3:
ID f4231055-4bf7-4f69-a39c-ef76de7359ae
Row id: 3, name: Charlie, email: charlie@example.com, age: 35
--------------------------------------------------


## 2.3 텍스트 분할

### 2.3.1 문서와 노드

In [30]:
from llama_index.core import Document

# 텍스트 데이터를 기반으로 문서 생성
document_text = "영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는 오랫동안 숨어 살던 남자가 있다는 반전이 있습니다. 이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다. 결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다."
document = Document(text=document_text)

# 메타데이터 추가
document.metadata = {'author': '영화 해설', 'subject': '기생충 줄거리'}

print(document)

Doc ID: d4bdc722-82ea-4b1c-a290-f92069bb9d10
Text: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 처음에는
평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는
오랫동안 숨어 살던 남자가 있다는 반전이 있습니다. 이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다.
결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다.


In [31]:
from llama_index.core.node_parser import SimpleNodeParser

parser = SimpleNodeParser(chunk_size=80, chunk_overlap=0)
nodes = parser.get_nodes_from_documents([document])

print("\n생성된 노드들")
for idx, node in enumerate(nodes, start=1):
    node.metadata = {'type': '영화 줄거리', 'genre': '드라마', 'node_id': idx}
    print(f"노드 {idx}: {node.text}")
    print(f"   메타데이터: {node.metadata}")


생성된 노드들
노드 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다.
   메타데이터: {'type': '영화 줄거리', 'genre': '드라마', 'node_id': 1}
노드 2: 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다.
   메타데이터: {'type': '영화 줄거리', 'genre': '드라마', 'node_id': 2}
노드 3: 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는 오랫동안 숨어 살던 남자가 있다는 반전이 있습니다.
   메타데이터: {'type': '영화 줄거리', 'genre': '드라마', 'node_id': 3}
노드 4: 이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다.
   메타데이터: {'type': '영화 줄거리', 'genre': '드라마', 'node_id': 4}
노드 5: 결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다.
   메타데이터: {'type': '영화 줄거리', 'genre': '드라마', 'node_id': 5}


In [32]:
from llama_index.core.node_parser import TokenTextSplitter
import tiktoken

# cl100k_base 토크나이저 로드
tokenizer = tiktoken.get_encoding("cl100k_base")

token_splitter = TokenTextSplitter(chunk_size=80, chunk_overlap=0)

chunks = token_splitter.split_text(document_text)

for i, chunk in enumerate(chunks):
    token_count = len(tokenizer.encode(chunk))  # 각 청크의 토큰 개수 계산
    print(f"청크 {i+1}: {chunk}\n[토큰 개수: {token_count}]\n")

print(f"총 생성된 청크 개수: {len(chunks)}")


청크 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, 이들의
[토큰 개수: 77]

청크 2: 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는 오랫동안 숨어 살던 남자가 있다는
[토큰 개수: 78]

청크 3: 반전이 있습니다. 이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다. 결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다.
[토큰 개수: 80]

총 생성된 청크 개수: 3


In [33]:
from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.llms.openai import OpenAI

# 한글 답변 설정
llm = OpenAI(api_key=os.environ["OPENAI_API_KEY"], model="gpt-4o-mini", system_prompt="반드시 한국어로 답변하세요.")
# Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0, system_prompt="항상 한국어로 답변하세요.")

# 문서 단위 검색을 위한 전체 문서 인덱스 생성
full_doc_index = VectorStoreIndex.from_documents([document], llm=llm)

# NodeParser를 사용하여 문서를 작은 의미 단위(Node)로 분할
# parser = SimpleNodeParser(chunk_size=100, chunk_overlap=0)
# nodes = parser.get_nodes_from_documents([document])

# 노드 단위 검색을 위한 인덱스 생성
node_index = VectorStoreIndex(nodes, llm=llm)

# 검색 비교
query_text = '이 영화의 반전은 무엇인가요?'

## 문서 단위 검색
print("\n문서 단위 검색 결과")
doc_query_engine = full_doc_index.as_query_engine()
doc_response = doc_query_engine.query(query_text)
print(f"문서 검색 응답: {doc_response.response}")
if doc_response.source_nodes:
    for idx, document in enumerate(doc_response.source_nodes, start=1):
        print(f"- 결과 {idx}: {document.node.text}")

## 노드 단위 검색
print("\n노드 단위 검색 결과")
node_query_engine = node_index.as_query_engine()
node_response = node_query_engine.query(query_text)
print(f"노드 검색 응답: {node_response.response}")
if node_response.source_nodes:
    for idx, document in enumerate(node_response.source_nodes, start=1):
        print(f"- 결과 노드 {idx}: {document.node.text}")



문서 단위 검색 결과
문서 검색 응답: The movie's twist reveals the existence of a secret basement in the wealthy Park family's house, where a man has been living in hiding for a long time. This unexpected revelation leads to a series of events that culminate in a tragic ending.
- 결과 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는 오랫동안 숨어 살던 남자가 있다는 반전이 있습니다. 이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다. 결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다.

노드 단위 검색 결과
노드 검색 응답: 이 영화의 반전은 박 사장네 집에 있는 비밀 지하실에 오랫동안 숨어 살던 남자가 있다는 것입니다.
- 결과 노드 1: 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는 오랫동안 숨어 살던 남자가 있다는 반전이 있습니다.
- 결과 노드 2: 결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다.


### 2.3.2 토큰 단위 분할

In [34]:
sample_text = "영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 그러나 이 영화의 반전은 지하실에서 시작됩니다."


token_splitter = TokenTextSplitter(
    chunk_size=50,
    chunk_overlap=10
)

token_chunks = token_splitter.split_text(sample_text)
print("=== 토큰 기반 분할 결과 ===")
for i, chunk in enumerate(token_chunks):
    print(f"Chunk {i + 1}:", chunk.strip(), "\n")



=== 토큰 기반 분할 결과 ===
Chunk 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 

Chunk 2: 취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 

Chunk 3: 쌓이며 긴장감이 점점 고조됩니다. 그러나 이 영화의 반전은 지하실에서 시작됩니다. 



### 2.3.3 문장 단위 분할

In [35]:
from llama_index.core.node_parser.text.sentence import SentenceSplitter

splitter = SentenceSplitter(chunk_size=50, chunk_overlap=0)

sentence_chunks = splitter.split_text(sample_text)
print("=== 문장 기반 분할 결과 ===")
for i, chunk in enumerate(sentence_chunks):
    print(f"Chunk {i + 1}:", chunk.strip(), "\n")


=== 문장 기반 분할 결과 ===
Chunk 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 

Chunk 2: 벌어지는 이야기입니다. 

Chunk 3: 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 

Chunk 4: 그러나 이 영화의 반전은 지하실에서 시작됩니다. 



### 2.3.4 의미 단위 분할

In [36]:
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding()

splitter = SemanticSplitterNodeParser(
    buffer_size=1,
    breakpoint_percentile_threshold=95,
    embed_model=embed_model
)

chunks = splitter.sentence_splitter(sample_text)
print("=== 의미 기반 분할 결과 ===")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:", chunk.strip(), "\n")


=== 의미 기반 분할 결과 ===
Chunk 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 

Chunk 2: 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 

Chunk 3: 그러나 이 영화의 반전은 지하실에서 시작됩니다. 



In [37]:
!pip install transformers -q
!pip install llama-index-embeddings-huggingface -q


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [38]:
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

splitter = SemanticSplitterNodeParser(
    buffer_size=1,
    breakpoint_percentile_threshold=95,
    embed_model=embed_model
)

chunks = splitter.sentence_splitter(sample_text)
print("=== 의미 기반 분할 결과 ===")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:", chunk.strip(), "\n")

=== 의미 기반 분할 결과 ===
Chunk 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 

Chunk 2: 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 

Chunk 3: 그러나 이 영화의 반전은 지하실에서 시작됩니다. 



## 2.4 인덱싱

### 2.4.2 벡터 저장소 인덱스

In [40]:
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader('data')
documents = reader.load_data()

# Document 객체를 전달하여 인덱스 생성
index = VectorStoreIndex.from_documents(documents)

### 2.4.3 Top-K 검색

In [41]:
# 상위 3개의 결과를 반환
query_engine = index.as_query_engine(similarity_top_k=3)

response = query_engine.query("고양이에게 수분 공급이 중요한 이유는?")

print("[검색된 상위 3개 문서]")

for idx, node in enumerate(response.source_nodes):
    print(f"\n[문서 {idx+1}]\n{node.text}")
print("\n[답변]")

print(response)

[검색된 상위 3개 문서]

[문서 1]
고양이는 물을 충분히 마셔야 합니다. 수분이 부족하면 신장 문제가 발생할 수 있습니다.
건식 사료보다 습식 사료가 수분 공급에 도움이 됩니다.

[문서 2]
고양이는 육식 동물입니다. 주로 고기, 생선, 그리고 가공된 고양이 사료를 먹습니다.
특히, 단백질이 풍부한 음식을 선호하며, 탄수화물 섭취는 적은 편입니다.

[문서 3]
고양이는 초콜릿, 양파, 마늘 같은 음식은 먹으면 안 됩니다.
특히, 초콜릿에 포함된 테오브로민 성분은 고양이에게 치명적일 수 있습니다.

[답변]
고양이에게 수분 공급이 중요한 이유는 신장 문제를 예방하기 위해서입니다.


## 2.5 저장하기

In [42]:
!pip install chromadb -q


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [43]:
import chromadb

# 크로마 클라이언트 초기화
db = chromadb.PersistentClient(path="./chroma_db")

In [44]:
# 컬렉션 생성하기
chroma_collection = db.get_or_create_collection("quickstart")

In [45]:
!pip install llama-index-vector-stores-chroma -q


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [46]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

# 크로마를 벡터 저장소로 지정
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [47]:
from llama_index.core import VectorStoreIndex

# 문서 리스트 (임의의 예시)
documents = [
    Document(text="Llama2 is a large language model developed by Meta."),
    Document(text="Chroma is an open-source vector store.")
]

# 문서를 벡터 스토어 인덱스로 저장
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

# 저장된 벡터 인덱스 데이터 저장
index.storage_context.persist(persist_dir="./index_data")

In [48]:
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

In [49]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

# 크로마 클라이언트 초기화
db = chromadb.PersistentClient(path="./chroma_db")

# 저장된 컬렉션을 다시 가져오기 (또는 생성하기)
chroma_collection = db.get_or_create_collection("quickstart")

# 저장된 크로마 벡터 스토어 설정
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# 벡터 스토어 인덱스에 문서를 저장 (데이터 임베딩 후 저장)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

# 인덱스 데이터를 로컬 디렉터리에 저장
index.storage_context.persist(persist_dir="./index_data")

# --- 이후 다시 데이터를 불러오고 쿼리 수행 ---
# 크로마 클라이언트 다시 초기화
db = chromadb.PersistentClient(path="./chroma_db")

# 저장된 컬렉션을 다시 가져오기
chroma_collection = db.get_or_create_collection("quickstart")

# 저장된 크로마 벡터 스토어 설정
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# 저장된 벡터 데이터를 이용해 인덱스를 다시 로드
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

# 쿼리 엔진 생성 및 쿼리 수행
query_engine = index.as_query_engine()
response = query_engine.query("What is Chroma?")
print(response)

Chroma is an open-source vector store.


## 2.6 쿼리

### 2.6.1 쿼리 엔진

In [51]:
query_engine = index.as_query_engine()
response = query_engine.query(
    "고객의 개인 정보를 참고하여 맞춤형 이메일을 작성해 주세요."
)
print(response)

I'm sorry, I cannot assist with that request.


### 2.6.2 검색

In [52]:
from llama_index.core.retrievers import VectorIndexRetriever

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5, # 상위 5개의 결과 반환
)

### 2.6.3 후처리

In [53]:
from llama_index.core.postprocessor import SimilarityPostprocessor

postprocessor = SimilarityPostprocessor(similarity_cutoff=0.7)
query_engine = index.as_query_engine(node_postprocessors=[postprocessor])

### 2.6.4 응답 합성

In [54]:
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

# 응답 합성기 설정
response_synthesizer = get_response_synthesizer(response_mode="compact")

# 쿼리 엔진 구성
query_engine = RetrieverQueryEngine.from_args(
    retriever=index.as_retriever(),
    response_synthesizer=response_synthesizer,
)

# 쿼리 실행
response = query_engine.query("Llama2란?")

### 2.6.5 커스터마이징

In [55]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# 문서를 기반으로 인덱스 생성
index = VectorStoreIndex.from_documents(documents)

In [56]:
# 검색기 설정 (상위 10개의 유사한 결과 반환)
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

In [57]:
# 응답 합성기 설정
response_synthesizer = get_response_synthesizer()

In [58]:
# 후처리 설정 (유사도 0.7 이상인 노드만 선택)
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.7)

In [59]:
# 쿼리 엔진
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

In [60]:
# 쿼리 실행 및 결과 출력
response = query_engine.query("모나리자 그림은 어디에 전시되어 있나요?")
print(response)

The Mona Lisa painting is displayed at the Louvre Museum in Paris, France.
